Reverse words
=============

Based on translate.py and on code from xxxxx.

To do
-----

- clean old code.
- remove save/restore model.
- do predict.
- check loss function.


In [23]:
"""Binary for training translation models and decoding from them.

See the following papers for more information on neural translation models.
 * http://arxiv.org/abs/1409.3215
 * http://arxiv.org/abs/1409.0473
 * http://arxiv.org/abs/1412.2007
"""

import math
import os
import random
import sys
import time

import numpy as np
import tensorflow as tf

from tensorflow.models.rnn.translate import data_utils
from tensorflow.models.rnn.translate import seq2seq_model


learning_rate = 0.5                  # "Learning rate."
learning_rate_decay_factor = 0.9     # "Learning rate decays by this much."
max_gradient_norm = 5.0              # "Clip gradients to this norm."
num_nodes = 128                      # "Size of each model layer.") # was 1024 
num_layers = 1                       # "Number of layers in the model.") # was 3

steps_per_checkpoint = 30            # "How many training steps to do per checkpoint.")
dodecode = False                     #, "Set to True for interactive decoding.")


# We use a number of buckets and pad to the closest one for efficiency.
# See seq2seq_model.Seq2SeqModel for details of how they work.
# _buckets = [(5, 10), (10, 15), (20, 25), (40, 50)]
_buckets = [(10, 11)]



In [7]:
import string

vocabulary_size = len(string.ascii_lowercase) # [a-z]
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
    if char in string.ascii_lowercase:
        return ord(char) - first_letter + 1
    else:
        print('Unexpected character: %s' % char)
        return 0
  
def id2char(dictid):
    if dictid > 0:
        return chr(dictid + first_letter - 1)
    else:
        return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character:  
Unexpected character: ï
1 26 0 0
a z  


In [8]:
def get_batch(size):
    encoder_inputs = [np.random.randint(1, vocabulary_size, size) for _ in xrange(batch_size)]
    decoder_inputs = [np.zeros(size+1, dtype = np.int32) for _ in xrange(batch_size)]
    weights = [np.ones(size+1, dtype = np.float32) for _ in xrange(batch_size)]
    for i in xrange(batch_size):
        r = random.randint(1, size-1)
        encoder_inputs[i][r:] = 0
        # Reverse the encoder input sequence, but leave a 0 at index 0 and at least one 0 at the end.
        # These are the GO and EOS markers.
        decoder_inputs[i][1:r+1] = encoder_inputs[i][:r][::-1]
        weights[i][r+1:] = 0.0
    return np.transpose(encoder_inputs), np.transpose(decoder_inputs), np.transpose(weights)


Bacthes are three matrices, encoder_inputs and decoder_inputs and weights. Weights are all zeroes.

encoder_inputs have one entry per column (a fake word) with one letter per row.

decoder_inputs (the "labels") are encoder_inputs reversed.

In [9]:
batch_size=20
b = get_batch(10)
print('e_i')
print(b[0])
print('d_i')
print(b[1])



e_i
[[15 22  6 10 19  7 14 13  4  3 20  6 13 18 20 11 22  8  1  1]
 [ 9 23 16 19 18 10  9 24 15 24 24 16 20  6 25 17  0 17 16  5]
 [10  4  8 23 24 19  7 16  7 24 10  4 18  9 20  4  0  6 21 13]
 [ 0  0  0 19 10  0  0  0 13 10  6 13  0 19 19 15  0  0 12 22]
 [ 0  0  0  2  0  0  0  0  9  9 13  0  0 18 18  3  0  0  0 18]
 [ 0  0  0 14  0  0  0  0  0  8  0  0  0 16  7 18  0  0  0 12]
 [ 0  0  0  7  0  0  0  0  0  8  0  0  0 14 18 17  0  0  0  6]
 [ 0  0  0  6  0  0  0  0  0 23  0  0  0  0  4  0  0  0  0  9]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]
d_i
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [10  4  8  6 10 19  7 16  9 23 13 13 18 14  4 17 22  6 12  9]
 [ 9 23 16  7 24 10  9 24 13  8  6  4 20 16 18 18  0 17 21  6]
 [15 22  6 14 18  7 14 13  7  8 10 16 13 18  7  3  0  8 16 12]
 [ 0  0  0  2 19  0  0  0 15  9 24  6  0 19 18 15  0  0  1 18]
 [ 0  0  0 19  0  0  0  0  4 10 20  0  0  9 19

In [10]:

def get_validation_batch(text, size):
    words = text.split()[:batch_size]
    encoder_inputs = [np.zeros(size, dtype = np.int32) for _ in xrange(batch_size)]
    for i, word in enumerate(words):
        l = min(len(word), size - 1)
        for j in xrange(l):
            encoder_inputs[i][j] = char2id(word[j])
    return np.transpose(encoder_inputs)

val_encoder_inputs = get_validation_batch("the quick brown fox jumps over the lazy dog", 10)
print(val_encoder_inputs)

[[20 17  2  6 10 15 20 12  4  0  0  0  0  0  0  0  0  0  0  0]
 [ 8 21 18 15 21 22  8  1 15  0  0  0  0  0  0  0  0  0  0  0]
 [ 5  9 15 24 13  5  5 26  7  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  3 23  0 16 18  0 25  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 11 14  0 19  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]


----------------------

Model
-----



In [24]:
  # model = seq2seq_model.Seq2SeqModel(source_vocab_size = vocabulary_size,
  #                                    target_vocab_size = vocabulary_size,
  #                                     buckets = [(10, 11)],
  #                                     size = num_nodes,
  #                                     num_layers = 1, # one encoding and one decoding LSTM
  #                                     max_gradient_norm = 5.0,
  #                                     batch_size = batch_size,
  #                                     learning_rate = 1.0,
  #                                     learning_rate_decay_factor = 0.9,
  #                                     use_lstm = True,
  #                                     forward_only = False)

def create_model(session, forward_only):
    """Create translation model and initialize or load parameters in session."""
    model = seq2seq_model.Seq2SeqModel(
        vocabulary_size, vocabulary_size, _buckets,
        num_nodes, num_layers, max_gradient_norm, batch_size,
        learning_rate, learning_rate_decay_factor,
        use_lstm = True, forward_only=forward_only)
    
    print("Created model with fresh parameters.")
    session.run(tf.initialize_all_variables())
    return model


In [25]:
num_steps = 3001


def train():

  with tf.Session() as sess:
    # Create model.
    print("Creating %d layers of %d units." % (num_layers, num_nodes))
    model = create_model(sess, False)

    # This is the training loop.
    step_time, loss = 0.0, 0.0
    current_step = 0
    previous_losses = []
    for _ in xrange(num_steps):

      # Get a batch and make a step.
      start_time = time.time()
      encoder_inputs, decoder_inputs, target_weights = get_batch(10)
      _, step_loss, _ = model.step(sess, encoder_inputs, decoder_inputs,
                                   target_weights, 0, False)
      step_time += (time.time() - start_time) / steps_per_checkpoint
      loss += step_loss / steps_per_checkpoint
      current_step += 1

      # Once in a while, we save checkpoint, print statistics, and run evals.
      if current_step % steps_per_checkpoint == 0:
        # Print statistics for the previous epoch.
        perplexity = math.exp(loss) if loss < 300 else float('inf')
        print ("global step %d learning rate %.4f step-time %.2f perplexity "
               "%.2f" % (model.global_step.eval(), model.learning_rate.eval(),
                         step_time, perplexity))
        # Decrease learning rate if no improvement was seen over last 3 times.
        if len(previous_losses) > 2 and loss > max(previous_losses[-3:]):
          sess.run(model.learning_rate_decay_op)
        previous_losses.append(loss)
        # Zero timer and loss.
        step_time, loss = 0.0, 0.0
        # Run evals on development set and print their perplexity.
        for bucket_id in xrange(len(_buckets)):
          if len(dev_set[bucket_id]) == 0:
            print("  eval: empty bucket %d" % (bucket_id))
            continue
          encoder_inputs, decoder_inputs, target_weights = model.get_batch(
              dev_set, bucket_id)
          _, eval_loss, _ = model.step(sess, encoder_inputs, decoder_inputs,
                                       target_weights, bucket_id, True)
          eval_ppx = math.exp(eval_loss) if eval_loss < 300 else float('inf')
          print("  eval: bucket %d perplexity %.2f" % (bucket_id, eval_ppx))
        sys.stdout.flush()


In [34]:
# should study this.

def evaluate_model(model, sess, val_encoder_inputs, output_size):
    val_decoder_inputs = np.zeros((output_size+1, batch_size,), dtype = np.int32)
    val_target_weights = np.zeros((output_size+1, batch_size,), dtype = np.float32)
    val_target_weights[0,:] = 1.0
    is_finished = np.full((batch_size,), False, dtype = np.bool_)
    for i in xrange(output_size):
        _, _, output_logits = model.step(sess, val_encoder_inputs, val_decoder_inputs, val_target_weights, bucket_id = 0, forward_only = True)
        p = np.argmax(output_logits[i], axis = 1)
        is_finished = np.logical_or(is_finished, p == 0)
        val_decoder_inputs[i,:] = (1 - is_finished) * p
        val_target_weights[i,:] = (1.0 - is_finished) * 1.0
    return val_decoder_inputs

In [26]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

train()


Creating 1 layers of 128 units.
Created model with fresh parameters.
global step 30 learning rate 0.5000 step-time 0.16 perplexity 17.05


NameError: name 'dev_set' is not defined

In [5]:
def decode():
  with tf.Session() as sess:
    # Create model and load parameters.
    model = create_model(sess, True)
    model.batch_size = 1  # We decode one sentence at a time.

    # Load vocabularies.
    en_vocab_path = os.path.join(FLAGS.data_dir,
                                 "vocab%d.en" % FLAGS.en_vocab_size)
    fr_vocab_path = os.path.join(FLAGS.data_dir,
                                 "vocab%d.fr" % FLAGS.fr_vocab_size)
    en_vocab, _ = data_utils.initialize_vocabulary(en_vocab_path)
    _, rev_fr_vocab = data_utils.initialize_vocabulary(fr_vocab_path)

    # Decode from standard input.
    sys.stdout.write("> ")
    sys.stdout.flush()
    sentence = sys.stdin.readline()
    while sentence:
      # Get token-ids for the input sentence.
      token_ids = data_utils.sentence_to_token_ids(tf.compat.as_bytes(sentence), en_vocab)
      # Which bucket does it belong to?
      bucket_id = min([b for b in xrange(len(_buckets))
                       if _buckets[b][0] > len(token_ids)])
      # Get a 1-element batch to feed the sentence to the model.
      encoder_inputs, decoder_inputs, target_weights = model.get_batch(
          {bucket_id: [(token_ids, [])]}, bucket_id)
      # Get output logits for the sentence.
      _, _, output_logits = model.step(sess, encoder_inputs, decoder_inputs,
                                       target_weights, bucket_id, True)
      # This is a greedy decoder - outputs are just argmaxes of output_logits.
      outputs = [int(np.argmax(logit, axis=1)) for logit in output_logits]
      # If there is an EOS symbol in outputs, cut them at that point.
      if data_utils.EOS_ID in outputs:
        outputs = outputs[:outputs.index(data_utils.EOS_ID)]
      # Print out French sentence corresponding to outputs.
      print(" ".join([tf.compat.as_str(rev_fr_vocab[output]) for output in outputs]))
      print("> ", end="")
      sys.stdout.flush()
      sentence = sys.stdin.readline()



In [16]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

def self_test():
    """Test the translation model."""
    
    with tf.Session() as sess:
        print("Self-test for neural translation model.")
        # Create model with vocabularies of 10, 2 small buckets, 2 layers of 32.
        model = seq2seq_model.Seq2SeqModel(10, 10, [(3, 3), (6, 6)], 32, 2,
                                       5.0, 32, 0.3, 0.99, num_samples=8)
        sess.run(tf.initialize_all_variables())

        # Fake data set for both the (3, 3) and (6, 6) bucket.
        data_set = ([([1, 1], [2, 2]), ([3, 3], [4]), ([5], [6])],
                    [([1, 1, 1, 1, 1], [2, 2, 2, 2, 2]), ([3, 3, 3], [5, 6])])
        for _ in xrange(5):  # Train the fake model for 5 steps.
            bucket_id = random.choice([0, 1])
            encoder_inputs, decoder_inputs, target_weights = model.get_batch(data_set, bucket_id)
            model.step(sess, encoder_inputs, decoder_inputs, target_weights, bucket_id, False)

self_test()

Self-test for neural translation model.
